# 3. Python implementation ANN from scratch
_Author: Maurice Snoeren_<br>
This notebook discusses an approach to implement an artificial neural network from scratch in Python. 
<img src="./images/ann2.png" width="600px" />

Before starting to code in Python, we need to define the approach. We need to take into account that we can configure the network in terms of total input, hidden and output nodes. Also the amount of hidden nodes should be configurable as well. The model that we will use is shown by the figure above. An important design aspect, is that the hidden layer contains the weight matrix $W_{hx}$ to calculate the output. This result that forward propagation can be easily done by iterating through the hidden layers. The output of the last hidden layer is the input for the output layer with weight $W_y$. 

To implement this functionality, we create different classes to do so. We define the class AHiddenLayer, that represent the hidden layer. The ANN defines the input, hidden and output layers and will provide the main functionality. To be able to configure for each node an activation function, we shall implement classes for this as well. So ... lets begin with the implementation of the ANN class. At this moment, we are able to calculate the output by implementing forward propagation. We will add a method to the class __forward propagation__. This method takes an input vector $x$ and return the output $y$.

In [ ]:
import numpy as np

class ANN:
    def __init__(self, num_input_nodes, num_output_nodes, output_activation): # construct the ANN object
        self.num_input_nodes   = num_input_nodes   # Hold the number of input nodes
        self.num_output_nodes  = num_output_nodes  # Hold the number of output nodes
        self.output_activation = output_activation # Hold the number of input nodes
        self.hidden_layers     = [] # Hold all the hidden_layer classes
        self.Wy                = np.random.rand(num_input_nodes, num_output_nodes) # Hold output layer weight matrix
        self.by                = np.zeros((1, num_output_nodes)) # Biases vector of the output nodes
        self.x                 = [] # Hold the input vector that is used for calculation
        self.zy                = [] # Hold the summation of the input and bias with the weights
        self.y                 = [] # hold the output vector

    def get_weight_matrix(self): # getter for the weight matrix
        return self.Wy

    def set_weight_matrix(self, Wy): # setter for the weight matrix
        self.Wy = Wy

    def get_biases_vector(self): # getter for the bias vector
        return self.by

    def set_biases_vector(self, by): # setter for the bias vector
        self.by = by

    def add_hidden_layer(self, hidden_layer): # add a new hidden layer to the ANN
        self.hidden_layers.append(hidden_layer) # Add the HiddenLayer class to the array
        self.Wy = np.random.rand(hidden_layer.num_hidden_nodes, self.num_output_nodes) # Re-initializes the output matrix
                                                                                       # based on number of hidden nodes
    def get_total_hidden_layers(self): # return how many hidden layers are configured
        return len(self.hidden_layers)

    def get_hidden_layer(self, i): # returns the hidden layer given the index (no checks performed!)
        return self.hidden_layers[i]
    
    def forward_propagation(self, x):
        self.x = x # store the input that we have used for the calculation

        if ( len(self.hidden_layers) == 0): # Within our design it is possible that no hidden layers exist!
            self.zy = np.dot( self.x, self.Wy ) + self.by
            self.y = self.output_activation.forward( self.zy )

        else: # when we have hidden layers, we iterate over these hidden layers
            input_vector = self.x # this input_vector is used to pass to the next layer
            for hidden_layer in self.hidden_layers:
                output_vector = hidden_layer.forward_propagation(input_vector) # the hidden layer class calculates the 
                                                                               # output based on the input.
                input_vector = output_vector # the next hidden layer will use the output of this hidden layer
            
            # calculate the output of the neural network using the output of the last hidden layer as input
            self.zy = np.dot( input_vector, self.Wy ) + self.by # first calculate the weight and bias result
            self.y  = self.output_activation.forward( self.zy ) # calculate the activation function

        return self.y # return the output activation vector of all the nodes

The ANN class is ready and is able to calculate its output. We need to implement the hidden layer class now. The forward propagation will be implemented immediatly.

In [ ]:
class ANNHiddenLayer:
    def __init__(self, num_input_nodes, num_hidden_nodes, activation):
        self.num_input_nodes  = num_input_nodes # number of nodes of the previous layer used as input
        self.num_hidden_nodes = num_hidden_nodes # number of hidden nodes of this layer to be used
        self.activation       = activation # the activation function that should be used for all hidden nodes
        self.x                = [] # Input vector of this hidden layer
        self.Wh               = np.random.rand(num_input_nodes, num_hidden_nodes) # Hidden weight matrix
        self.bh               = np.zeros((1, num_hidden_nodes)) # Biases vector of the hidden layer
        self.zh               = [] # Hold the summation of the input and bias with the weights
        self.h                = [] # Hold the output vector of this hidden layer

    def get_weight_matrix(self): # getter for the weight matrix of the hidden layer
        return self.Wh

    def set_weight_matrix(self, Wh): # setter for the weight matrix of the hidden layer
        self.Wh = Wh

    def get_biases_vector(self): # getter for the biases vector of the hidden layer
        return self.bh

    def set_biases_vector(self, bh): # setter for the biases vector of the hidden layer
        self.bh = bh

    def forward_propagation(self, x):
        self.x = x # store the input that is used for the calculation
        self.zh = np.dot(x, self.Wh) + self.bh # first calculate the weight and bias result
        self.h  = self.activation.forward( self.zh ) # calculate the activation function
        
        return self.h # return the output activation vector of all the nodes

It is going allright! We still mis the activation function class. For this example we will only implement the sigmoid activation function. This class should implement the method __forward__ that executes the activation function using the given vector.

In [ ]:
class ANNSigmoidActivation:
    def forward(self, input_vector):
        return 1/(1 + np.exp(-input_vector)) 

Everything is in place and we can now try the class and see whether the calculation is done correctly.

In [ ]:
sa  = ANNSigmoidActivation() # construct the Sigmoid activation function

ann = ANN(4, 2, sa) # create an ANN with four input nodes and two output nodes. The output nodes get the sigmoid
                    # activation function.
    
ann.add_hidden_layer(ANNHiddenLayer(4, 10, sa)) # add a hidden layer with ten nodes, the input is four due to the
                                                  # total of number of input nodes x, defines by the ANN.
    
ann.add_hidden_layer(ANNHiddenLayer(10, 10, sa)) # create another hidden layer with ten node, the input is ten nodes
                                                   # due to the input nodes of the previoud hidden layer of ten.

x = np.array([[0.1, 0.1, 0.1, 0.1]]) # create an example input vector x

print( "output: " + str(ann.forward_propagation(x)) ) # print the output of the network